In this trial, I am performing Domain Adaptatation Techniques.

In [423]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Model
import time
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import keras as K
from tensorflow import keras
#from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from keras.layers import Dense
from tensorflow.keras.preprocessing import image


In [424]:

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [425]:
#defining training and testing datasets
(x_train, y_train), (x_test,y_test) = datasets.mnist.load_data()

In [426]:
#adding 4th dimension as 1 to declare as grayscale image
#normalizing the images
x_train = x_train.reshape((60000, 28, 28, 1))
x_train = x_train / 255.0
x_test = x_test.reshape((10000, 28, 28, 1))
x_test = x_test / 255.0

In [427]:
# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv2d_1', trainable = False),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_2', trainable = False),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_3', trainable = False),
    tf.keras.layers.Flatten(name='flatten'),
    # tf.keras.layers.Dense(128, activation='relu', name='dense_1'),
    tf.keras.layers.Dense(47, activation='softmax', name='output')
])

In [428]:
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [429]:
# Train the model and display the activations after each epoch
start_time = time.time()
history = model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))
end_time = time.time()

1875/1875 [==============================] - 13s 7ms/step - loss: 0.3257 - accuracy: 0.9189 - val_loss: 0.1766 - val_accuracy: 0.9517


In [430]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

313/313 - 1s - loss: 0.1766 - accuracy: 0.9517 - 1s/epoch - 5ms/step
Test accuracy: 0.95169997215271
Time elapsed:  12.85428500175476


In [431]:
# # %pip install emnist
# import emnist
# from emnist import extract_training_samples
# from emnist import extract_test_samples

from sklearn.model_selection import train_test_split

# Load Fashion MNIST dataset
(test_img, test_labels), (_, _) = datasets.fashion_mnist.load_data()

In [432]:
# emnist.list_datasets()

In [433]:
# #using EMNIST (Extended MNIST) dataset
# target_img,target_labels = extract_training_samples('digits')
# #test_img,test_labels = extract_test_samples('letters')

In [434]:
#reshaping the datasets
test_img = test_img.reshape((60000, 28, 28, 1))
test_img = test_img / 255.0

In [435]:
# Define a function to generate dummy labels
def generate_dummy_labels(num_samples, num_classes):
    return np.random.randint(0, num_classes, num_samples)

# Assuming num_classes in EMNIST is 26 for letters
num_classes_emnist = 10

# Generate dummy labels for EMNIST
dummy_labels = generate_dummy_labels(test_img.shape[0], num_classes_emnist)

In [436]:
#testing with no labels
test_loss, test_acc = model.evaluate(test_img, dummy_labels, verbose=2)

1875/1875 - 9s - loss: 6.1756 - accuracy: 0.1001 - 9s/epoch - 5ms/step


Here, we see the accuracy is pretty poor.

In [437]:
#testing with labels
test_loss, test_acc = model.evaluate(test_img, test_labels, verbose=2)

1875/1875 - 9s - loss: 6.1824 - accuracy: 0.0728 - 9s/epoch - 5ms/step


Here, we see the accuracy is still not great.

In [438]:
#predicting the label of a number based image
# Load and preprocess the input image
img_path = 'num1.png'
img = image.load_img(img_path, target_size=(28, 28), color_mode='grayscale')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.0

# Predict the label
predictions = model.predict(x)
predicted_label = np.argmax(predictions[0])

print(f'Predicted label: {predicted_label}')

1/1 [==============================] - 0s 41ms/step
Predicted label: 3


Let's build a DANN now and examine the accuracy

In [439]:
# Define the Feature Extractor
def build_feature_extractor(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape,name='conv2d_1'),
        layers.Conv2D(32, (3,3), activation='relu',name='conv2d_2'),
        layers.Conv2D(32, (3,3), activation='relu',name='conv2d_3'),
        layers.Flatten(name='flatten')
    ])
    return model

In [440]:
# Define the Label Predictor
def build_label_predictor(num_classes):
    model = models.Sequential([
        layers.Dense(num_classes, activation='softmax',name='output')
    ])
    return model


In [441]:
# Define the Domain Predictor
def build_domain_predictor():
    model = models.Sequential([
        layers.Dense(64, activation='relu',name='dense_1'),
        layers.Dense(1, activation='sigmoid',name='dense_2')
    ])
    return model

In [442]:
# Build the complete DANN model
def build_dann(input_shape, num_classes):
    feature_extractor = build_feature_extractor(input_shape)
    label_predictor = build_label_predictor(num_classes)
    domain_predictor = build_domain_predictor()

    # Define inputs
    input_data = layers.Input(shape=input_shape)
    label = layers.Input(shape=(num_classes,))
    domain_label = layers.Input(shape=(1,))
    
    # Feature extractor output
    feature_output = feature_extractor(input_data)
    
    # Label prediction branch
    label_output = label_predictor(feature_output)
    
    # Domain prediction branch
    domain_output = domain_predictor(feature_output)
    
    dann_model = models.Model(inputs=[input_data, label, domain_label], 
                              outputs=[label_output, domain_output])
    
    return dann_model

In [443]:
# Define loss functions
def label_loss(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred)

def domain_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

In [444]:
# Build and compile the DANN model
input_shape = (28, 28, 1)
num_classes = 10

dann_model = build_dann(input_shape, num_classes)

dann_model.compile(optimizer='Adam',
                   loss=[label_loss, domain_loss],
                   loss_weights=[1.0,0.1])


In [445]:
# Prepare domain labels
domain_label_source = np.zeros((x_train.shape[0], 1))  # Source domain label is 0
domain_label_target = np.ones((test_img.shape[0], 1))   # Target domain label is 1


In [446]:
# One-hot encode the train labels
#shape of y_train is (6000,) initially despite having 10 classes
# it means that it contains class labels as integers rather than being one-hot encoded.
y_train = tf.keras.utils.to_categorical(y_train, 10) 

In [447]:
# Concatenate source and target data
x_combined = np.vstack((x_train, test_img))
y_combined = np.vstack((y_train, np.zeros((y_train.shape))))  # Labels for target domain can be ignored
domain_labels_combined = np.vstack((domain_label_source, domain_label_target))

In [448]:
# Train the DANN

start_time = time.time()

dann_model.fit([x_combined,y_combined,domain_labels_combined], 
               [x_combined,y_combined], 
               epochs=1)

end_time = time.time()

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/var/folders/b8/466710_d78q31z8wgrvw2cc00000gn/T/ipykernel_84611/2340007477.py", line 3, in label_loss  *
        return tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/losses.py", line 2122, in categorical_crossentropy
        return backend.categorical_crossentropy(
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/backend.py", line 5560, in categorical_crossentropy
        target.shape.assert_is_compatible_with(output.shape)

    ValueError: Shapes (32, 28, 28, 1) and (32, 10) are incompatible
